From the requests library, there are many functions that will help us. $request.head()$ and $request.get()$ seem the most useful. I will illustrate below with a bad tweet: 

In [1]:
# Using 'https://t.co/I5HajUrJ61'
import requests as rq 
r = rq.head('https://t.co/I5HajUrJ61'); 
print(r)

<Response [301]>


Using head, we find our response code. 301. From wikipedia: 

The HTTP response status code 301 Moved Permanently is used for permanent URL redirection, meaning current links or records using the URL that the response is received for should be updated.

Using 301 we can computationaly identify when the link is "bad".  

Futher more, to find the unshortened the url we have: 

In [5]:
extendedurl = rq.get('https://t.co/I5HajUrJ61')
print(extendedurl)
extendedurl.url

<Response [200]>


'https://twitter.com/i/web/status/1188083344724185088'

The HTTP code 200 implies that the request was successful. That is the new extendedurl points to the correct location. 
Lastly using .url will print our new extended url.


Branden's idea: Create a something that will check for the 301 condition if 301 is true, replace. 

In order to accomplish this we need to return only the code with $response.status$_$code$

In [3]:
def unshorten_url(url):
    r = rq.head(url);
    if r.status_code == 301: 
        r = rq.get(url); # Pull the full url
    return r.url # returns the url 

In [4]:
unshorten_url('https://t.co/I5HajUrJ61')

'https://twitter.com/i/web/status/1188083344724185088'

While this works when the code is 301, but there are many codes that will indicate a "bad" url. A few examples are 302, 400, and 402. Further research indicate a scheme. From wiki:

1xx informational response – the request was received, continuing process

2xx successful – the request was successfully received, understood and accepted

3xx redirection – further action needs to be taken in order to complete the request

4xx client error – the request contains bad syntax or cannot be fulfilled

5xx server error – the server failed to fulfill an apparently valid request

Thus we should modify our code for any 300 case(possibly the others?). 

In [8]:
def unshorten_url(url):
    r = rq.head(url);
    if int(r.status_code/100) ==  3: # int() will change any float that leads in 3 to an integer.  
        r = rq.get(url); # Pull the full url
    return r.url # returns the url 

unshorten_url('https://t.co/I5HajUrJ61')

'https://twitter.com/i/web/status/1188083344724185088'

Our data already contains the bad urls, thus we should not have to worry about "bandwith". 

Using the defintion in Parker's code we encounter an issue: 


In [15]:
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
#import mpld3

df = pd.read_csv(r"C:\Users\JungleBook\Desktop\DS Club\OpioidTwitter-master\data\opioid_tweets.csv")



with open("bad_tweets.txt", "r") as f:
    ids = [i.strip() for i in f.read().split(",")] 

for item in df[df["id"].isin(ids)].iterrows():
    print()
    print("ID: {}".format(item[1]["id"]))
    print()
    print(unshorten_url(item[1]["content"]) )
# The isssue is that item[1]["Content] contains the text of tweet the url"
    print()
    print("User: {}".format(item[1]["user_name"]))
    print("_____________________________________________________________________________________________")


FileNotFoundError: [Errno 2] No such file or directory: 'bad_tweets.txt'

While I would love to show you that our issue is with $item[1]["content"]$ containg a tweet, then url. I cannot open my text file in Juypter Notebooks. Any help would be apprectiated. 